In [14]:
from lxml import html
import requests
import os

In [2]:
url = "https://dumps.wikimedia.org/other/pagecounts-raw/2015/2015-12/"
page = requests.get(url)
tree = html.fromstring(page.content)

In [3]:
pcounts = tree.xpath('//ul/li/a/@href')

In [4]:
def fetch_gz(fname):
    # partial page count url
    ppc_url = 'https://dumps.wikimedia.org/other/pagecounts-raw/2015/2015-12/'
    print("fetching: "+ fname)
    pc_url = ppc_url+fname
    r = requests.get(pc_url, stream=True)
    with open(fname, 'wb') as f:
            for chunk in r.iter_content(chunk_size=1024):
                if chunk:
                    f.write(chunk)
            f.flush()

In [5]:
from hdfs3 import HDFileSystem
hdfs = HDFileSystem()

In [15]:
def fetch_gz(fname):
    # partial page count url
    ppc_url = 'https://dumps.wikimedia.org/other/pagecounts-raw/2015/2015-12/'
    print("fetching: "+ fname)
    pc_url = ppc_url+fname
    r = requests.get(pc_url, stream=True)
    fname_path = os.path.join('/tmp','wiki',fname)
    with hdfs.open(fname_path, 'w') as f:
            for chunk in r.iter_content(chunk_size=1024):
                if chunk:
                    f.write(chunk)

In [9]:
from distributed import Executor
executor = Executor('127.0.0.1:8786')

In [28]:
executor.restart()

In [36]:
x = executor.map(fetch_gz, pcounts)

In [48]:
x

[<Future: status: finished, key: fetch_gz-34fff84b1e07c2040af8160c41b541f4>,
 <Future: status: finished, key: fetch_gz-671b32743272177ac9b7e3cfe0bd764a>,
 <Future: status: finished, key: fetch_gz-c765f0eabbbd766e677d582d8f0db5ef>,
 <Future: status: finished, key: fetch_gz-0daf884f399829d9266dcf90c374eda8>,
 <Future: status: pending, key: fetch_gz-e417b921d73dc5ffaca37f848f3bc2e0>,
 <Future: status: pending, key: fetch_gz-f792456935605ef5ef0091da361ff76e>,
 <Future: status: pending, key: fetch_gz-20c7dfab8b71e5c62b6aa7a4f3810f0a>,
 <Future: status: pending, key: fetch_gz-be27f57604d65dd7bf8846347f3a495f>,
 <Future: status: finished, key: fetch_gz-8d9887760eae8931798c412a2d3009c2>,
 <Future: status: pending, key: fetch_gz-2e0cffd2801e05a61bd6c93283f55ad7>,
 <Future: status: pending, key: fetch_gz-dd38fedc8f6fe576bb8738d4340c2944>,
 <Future: status: pending, key: fetch_gz-002e28d792cb8eb16c8024b5cb08f311>,
 <Future: status: pending, key: fetch_gz-5274c8385a05a150d5a00d020b39ccb4>,
 <Futur

In [31]:
from distributed import progress

In [57]:
progress(x)

In [49]:
hdfs.ls('/tmp/wiki/')

[{'block_size': 0,
  'group': 'hadoop',
  'kind': 'directory',
  'last_access': 0,
  'last_mod': 1454391495,
  'name': '/tmp/wiki/data',
  'owner': 'ubuntu',
  'permissions': 493,
  'replication': 0,
  'size': 0},
 {'block_size': 67108864,
  'group': 'hadoop',
  'kind': 'file',
  'last_access': 1454391566,
  'last_mod': 1454391612,
  'name': '/tmp/wiki/pagecounts-20151201-000000.gz',
  'owner': 'root',
  'permissions': 511,
  'replication': 1,
  'size': 94679147},
 {'block_size': 67108864,
  'group': 'hadoop',
  'kind': 'file',
  'last_access': 1454391566,
  'last_mod': 1454391611,
  'name': '/tmp/wiki/pagecounts-20151201-010000.gz',
  'owner': 'root',
  'permissions': 511,
  'replication': 1,
  'size': 93842592},
 {'block_size': 67108864,
  'group': 'hadoop',
  'kind': 'file',
  'last_access': 1454391566,
  'last_mod': 1454391608,
  'name': '/tmp/wiki/pagecounts-20151201-020000.gz',
  'owner': 'root',
  'permissions': 511,
  'replication': 1,
  'size': 88012500},
 {'block_size': 67108

In [43]:
from gzip import GzipFile
with hdfs.open('/tmp/wiki/pagecounts-20151229-070000.gz') as f:
    g = GzipFile(fileobj=f)
    print(g.read(1000))

TypeError: seek() takes exactly 2 arguments (3 given)

In [55]:
with hdfs.open('/tmp/wiki/projectcounts-20151218-000000') as f:
    f.seek(10000)
    print(f.read(1000))

d - 4764 43560604
ko.mw - 28479 373869046
ko.n - 44 546169
ko.q - 64 539470
ko.s - 563 14726576
ko.v - 58 505167
koi - 134 3782130
koi.mw - 16 144293
kr - 26 241290
kr.mw - 1 13112
krc - 271 7345543
krc.mw - 34 310887
ks - 52 789731
ks.b - 2 10571
ks.d - 16 84240
ks.mw - 15 108180
ks.q - 2 10696
ksh - 303 5971905
ksh.mw - 282 1651049
ku - 1065 17919762
ku.b - 18 124789
ku.d - 3537 23397353
ku.mw - 1211 6216555
ku.q - 40 348650
kv - 413 11204383
kv.mw - 30 223379
kw - 233 4945289
kw.d - 18 93639
kw.mw - 53 352874
kw.q - 4 22708
ky - 588 10164717
ky.b - 12 104107
ky.d - 180 1391629
ky.mw - 263 2061768
ky.q - 6 41306
la - 3721 56354462
la.b - 7 81998
la.d - 260 2338494
la.mw - 1397 9223522
la.q - 29 165444
la.s - 579 8101638
la.v - 1 904
lad - 1161 13967342
lad.mw - 35 231472
lb - 1807 28489550
lb.d - 180 1252750
lb.mw - 321 2846966
lb.q - 10 101628
lbe - 95 1783035
lbe.mw - 11 58608
lez - 193 6453573
lez.mw - 89 895400
lg - 1956 15279271
lg.mw - 2 7704
li - 679 11054468
li.b - 4 30498
li

In [56]:
hdfs.df()

{'capacity': 1267687342080, 'free': 98, 'used': 13108375014}

In [46]:
hdfs.exists('/tmp/wiki/pagecounts-20151229-070000')

False

## Notes

- 'wb' vs 'w'
- flush on close ?
- autocreate directories: /tmp/wiki/pagecounts-20151201-020000.gz
- seek offset whence -- gzip https://docs.python.org/2.4/lib/bltin-file-objects.html
- progress(notebook=False) in the notebook doesn't work